In [1]:
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams
import numpy as np
import pandas as pd
from qdrant_client.models import PointStruct
from qdrant_client.models import Filter, FieldCondition, MatchValue
import os

In [2]:
dense_model = SentenceTransformer("BAAI/bge-m3")
print(dense_model)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 8192, 'do_lower_case': False, 'architecture': 'XLMRobertaModel'})
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)


In [3]:
client_qdrant = QdrantClient(path=r"E:\SUDO_FINAL\RAG2\app\data\qdrant_storage")

In [29]:
query = "trái cây"
query_vec = dense_model.encode([query], convert_to_numpy=True).astype("float32")[0]

results = client_qdrant.query_points(
    collection_name="sign_vectors",
    query=query_vec,
    limit=10
)


for point in results.points:
    print(f"ID: {point.id}")
    print(f"Score: {point.score}")
    print(f"Payload: {point.payload}")
    print("-" * 40)

ID: 1264
Score: 0.5179114864069325
Payload: {'word': 'cây ăn quả', 'region': 'miền Trung', 'video': 'https://storage.googleapis.com/vsl-data-bucket/videos/Videos/W00456T.mp4', 'description': 'ERROR: Max Retries Exceeded', 'full_description': "Ký hiệu của từ 'cây ăn quả' (miền Trung): ERROR: Max Retries Exceeded"}
----------------------------------------
ID: 1255
Score: 0.5006514387798227
Payload: {'word': 'cây ăn quả', 'region': 'miền Nam', 'video': 'https://storage.googleapis.com/vsl-data-bucket/videos/Videos/W00456N.mp4', 'description': '1. Tay trái mở, lòng bàn tay úp, các ngón tay khép, hướng về phía trước. 2. Tay phải mở, các ngón tay xoè, lòng bàn tay hướng sang trái, đặt lên mu bàn tay trái, đưa tay phải lên trên đồng thời các ngón tay phải rung nhẹ. 3. Tay phải chụm lại, lòng bàn tay hướng sang trái, đặt sát các đầu ngón tay vào khoé miệng phải, giữ nguyên cánh tay, lắc cổ tay qua - lại 2 - 3 lần.', 'full_description': "Ký hiệu của từ 'cây ăn quả' (miền Nam): 1. Tay trái mở, lò

# LLM

In [42]:
import google.generativeai as genai
import os
from dotenv import load_dotenv

In [34]:
# Load .env
load_dotenv()
api_key = os.getenv("GEMINI_API_KEY")

In [44]:
genai.configure(api_key=api_key)
model = genai.GenerativeModel("gemini-2.0-flash")

user_query = "Tôi chuẩn bị đi ăn tối ở nhà hàng với bạn bè vào cuối tuần này. Bạn có thể giúp tôi học một số từ ngữ ký hiệu liên quan đến việc đi ăn không?"

intent_prompt = f"""Hãy trích xuất ra các từ khóa quan trọng mô tả ý định của câu sau, cách nhau bởi dấu phẩy: "{user_query}" """

resp = model.generate_content(intent_prompt)

resp.text

'ăn tối, nhà hàng, bạn bè, cuối tuần, từ ngữ ký hiệu, đi ăn\n'

In [45]:
keywords_raw = resp.text.strip() if hasattr(resp, "text") else str(resp)
keywords = [kw.strip() for kw in keywords_raw.split(",") if kw.strip()] or [user_query]
print("Extracted Keywords:", keywords)

Extracted Keywords: ['ăn tối', 'nhà hàng', 'bạn bè', 'cuối tuần', 'từ ngữ ký hiệu', 'đi ăn']


In [46]:
results_by_keyword = {}

for kw in keywords:
    vec = dense_model.encode([kw], convert_to_numpy=True).astype("float32")[0]
    
    hits = client_qdrant.query_points(
        collection_name="sign_vectors",
        query=vec,
        limit=10
    )
    
    keyword_results = [
        {
            "score": float(h.score),
            "payload": h.payload
        }
        for h in hits.points 
    ]
    
    results_by_keyword[kw] = {
        "total_results": len(keyword_results),
        "results": keyword_results
    }

results_by_keyword

{'ăn tối': {'total_results': 10,
  'results': [{'score': 0.5350244026059308,
    'payload': {'word': 'tiệc',
     'region': None,
     'video': 'https://storage.googleapis.com/vsl-data-bucket/videos/Videos/W03393.mp4',
     'description': 'Hai tay mở, lòng bàn tay hướng vào nhau, đầu ngón tay hướng lên trên, đặt trước ngực. Đưa đồng thời hai tay sang hai bên, lòng bàn tay hướng lên trên.',
     'full_description': "Ký hiệu của từ 'tiệc': Hai tay mở, lòng bàn tay hướng vào nhau, đầu ngón tay hướng lên trên, đặt trước ngực. Đưa đồng thời hai tay sang hai bên, lòng bàn tay hướng lên trên."}},
   {'score': 0.5138562505399095,
    'payload': {'word': 'đồ ăn',
     'region': None,
     'video': 'https://storage.googleapis.com/vsl-data-bucket/videos/Videos/D0444.mp4',
     'description': 'Tay phải chụm lại, lòng bàn tay hướng lên trên, đặt trước miệng, đưa tay lên - xuống 2 - 3 lần.',
     'full_description': "Ký hiệu của từ 'đồ ăn': Tay phải chụm lại, lòng bàn tay hướng lên trên, đặt trước m